## Taller 1 Visualizacion de Datos

Diego Silva Madariaga
Pablo Ibañez Pardo

In [1]:
# Instala las librerías: praw (API de Reddit), pandas (análisis de datos), duckdb (base de datos analítica).
!pip install praw pandas duckdb



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\diego\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
!pip install datetime


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\diego\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
#importar las librerias
import pandas as pd
import praw
import datetime
import numpy as numpy

# Librería que permite tratar dataframes como tablas sql
import duckdb

In [4]:
#funcion que guarda la credencial 
reddit = praw.Reddit(
                                    client_id="BlITQKHTmAdDpt-RuC3Xlw", 
                                    client_secret="gVHTWcDqVl_MJsYcMl_UadJcLFw4lA", 
                                    user_agent="banshhe02"
                                    )

In [5]:
# los subreddits a usar 
republicachile="RepublicadeChile"
mangacollectors="MangaCollectors"
manga="manga"
anime="anime"

#funcion obtiene informacion de un subreddit y la muestra
def info_subreddit(subreddit):
    print(f"Nombre completo: r/{subreddit.display_name}")
    print(f"Título: {subreddit.title}")
    print(f"Descripción: {subreddit.public_description}")
    print(f"Suscriptores: {subreddit.subscribers:,}")
    print(f"Usuarios activos: {subreddit.active_user_count:,}")
    print(f"Fecha de creación: {datetime.datetime.fromtimestamp(subreddit.created_utc).strftime('%Y-%m-%d %H:%M:%S')}")

## Subreddits usados

Para este taller se usaron los subreddits republica de Chile,anime ,manga y manga collectors.

El subrredit republica de Chile lo elegi por que en el se suelen discutir muchos temas de la actualidad del pais,tanto temas serios como casuales por lo que es una buena fuente para ver los temas que mas se hablan en el pais.
En cuanto a los otros 3 subreddit fueron elegidos primero por que el anime y el manga son unos hobbies que cada vez tienen mas seguidores en el pais,por lo que tomar al obtener la informacion de estos subreddit se puede saber datos relevantes como cuales son las series de anime y manga mas populares

In [6]:
#toma la informacion de los subreddit y la muestra usando la funcion definida arriba(info_subreddit)
info_subreddit(reddit.subreddit("RepublicadeChile"))
info_subreddit(reddit.subreddit("MangaCollectors"))

Nombre completo: r/RepublicadeChile
Título: El sub más libre de Chile
Descripción: Bienvenidos a r/RepublicadeChile, el espacio virtual más libre para chilenos e interesados en Chile de Reddit. Explora y comparte temas culturales, tradicionales y de interés general. Las publicaciones en inglés también son bienvenidas.
Suscriptores: 90,098
Usuarios activos: 42
Fecha de creación: 2018-10-29 19:35:36
Nombre completo: r/MangaCollectors
Título: Manga Collectors
Descripción: 
Suscriptores: 1,906,636
Usuarios activos: 22
Fecha de creación: 2017-10-15 19:03:23


In [7]:
info_subreddit(reddit.subreddit("manga"))

info_subreddit(reddit.subreddit("anime"))

Nombre completo: r/manga
Título: /r/manga: manga, on reddit.
Descripción: Everything and anything manga!  (manhwa/manhua is okay too!)

Discuss weekly chapters, find/recommend a new series to read, post a picture of your collection, lurk, etc!
Suscriptores: 4,727,080
Usuarios activos: 1,039
Fecha de creación: 2008-08-09 16:35:11
Nombre completo: r/anime
Título: /r/Anime
Descripción: Reddit's premier anime community.
Suscriptores: 13,057,298
Usuarios activos: 1,403
Fecha de creación: 2008-01-25 02:10:16


In [8]:
#creamos la estructura para almacenar los datos que extraigamos, tal como indica la pauta
datos = {
    'id_post': [],
    'titulo_post': [],
    "fecha_post": [],
    "autor_post": [],
    "URL": [],
    "numero_comentarios": [],
    "texto_post": [],
    "tiene_texto": [],
    "puntaje": [],     
    "upvotes": [],    
    "downvotes": []   
}

comentarios = {
    'id_post': [],
    'id_comentario': [],
    'autor_comentario': [],
    'texto_comentario': [],
    'fecha_comentario': []
}

In [9]:
#Funcion que extrae la informacion de un post de un subreddit y sus 10 primeros comentarios
def extraccion_info_post(post):
    fecha = datetime.datetime.fromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S')
    autor = str(post.author) if post.author else "ELIMINADO"
    tiene_texto = "Sí" if post.selftext else "No"

    datos['id_post'].append(post.id)
    datos['titulo_post'].append(post.title)
    datos['fecha_post'].append(fecha)
    datos['autor_post'].append(autor)
    datos['URL'].append(post.url)
    datos['numero_comentarios'].append(post.num_comments)
    datos['texto_post'].append(post.selftext)
    datos['tiene_texto'].append(tiene_texto)
    datos['puntaje'].append(post.score)  
    datos['upvotes'].append(post.ups)    
    datos['downvotes'].append(post.downs)  


    post.comments.replace_more(limit=0)  
    for comment in post.comments[:10]: #10 primeros comentarios 
        if not comment.stickied:  # Ignorar comentarios fijados (usualmente son de moderadores)
            comentarios['id_post'].append(post.id)
            comentarios['id_comentario'].append(comment.id)
            comentarios['autor_comentario'].append(str(comment.author) if comment.author else "ELIMINADO")
            comentarios['texto_comentario'].append(comment.body)
            comentarios['fecha_comentario'].append(
                datetime.datetime.fromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S')
            )

In [10]:
#En esta funcion seleccionamos un subreddit y se obtiene las 100 publicaciones mas votadas durante el mes y usa la funcion de extraccion anterior para extraer y guardar la informacion

subreddits_a_procesar = ["RepublicadeChile", "MangaCollectors", "manga", "anime"]
limit_por_subreddit = 100
time_filter_period = 'month'

for subreddit_nombre in subreddits_a_procesar:
    sub = reddit.subreddit(subreddit_nombre)
    for submission in sub.top(limit=limit_por_subreddit, time_filter=time_filter_period):
        extraccion_info_post(submission)

### Dataframe

In [11]:
#Se usa la libreria pandas para transformar los datos recopilados en un dataframe
df_posts = pd.DataFrame(datos)
df_comentarios = pd.DataFrame(comentarios)

In [12]:
# Une df_posts y df_comentarios en 'id_post' usando sql DuckDB y crea un nuevo DataFrame.
df_duckdb=duckdb.sql("""SELECT * FROM df_posts dp JOIN df_comentarios  dc on dc.id_post=dp.id_post""").df()

In [13]:
#se muestra el resultado de la union de los dataframe
df_duckdb

,id_post,titulo_post,fecha_post,autor_post,URL,numero_comentarios,texto_post,tiene_texto,puntaje,upvotes,downvotes,id_post_1,id_comentario,autor_comentario,texto_comentario,fecha_comentario
0,1jmdog7,Cuando te digan que Chile es xenofobico muestr...,2025-03-29 00:40:52,Sensitive-Play-4770,https://v.redd.it/y9r623zsujre1,153,,No,1172,1172,0,1jmdog7,mkbnlz4,GreyColdFlesh,Perú es clave,2025-03-29 04:32:13
1,1jmdog7,Cuando te digan que Chile es xenofobico muestr...,2025-03-29 00:40:52,Sensitive-Play-4770,https://v.redd.it/y9r623zsujre1,153,,No,1172,1172,0,1jmdog7,mkb3kjl,Lactantedelpico,"Me caen bien los peruanos, odian a los venecos...",2025-03-29 01:19:06
2,1jmdog7,Cuando te digan que Chile es xenofobico muestr...,2025-03-29 00:40:52,Sensitive-Play-4770,https://v.redd.it/y9r623zsujre1,153,,No,1172,1172,0,1jmdog7,mkbweg7,-Red02-,"Peruanos y Chilenos, separados por los Bolivia...",2025-03-29 06:12:33
3,1jmdog7,Cuando te digan que Chile es xenofobico muestr...,2025-03-29 00:40:52,Sensitive-Play-4770,https://v.redd.it/y9r623zsujre1,153,,No,1172,1172,0,1jmdog7,mkb01x4,ShutUp_Pls,De algo que sirva su caga de bandera\n\n![gif]...,2025-03-29 00:52:34
4,1jmdog7,Cuando te digan que Chile es xenofobico muestr...,2025-03-29 00:40:52,Sensitive-Play-4770,https://v.redd.it/y9r623zsujre1,153,,No,1172,1172,0,1jmdog7,mkbh3bd,CarmesiRojo,En Perú literalmente los han salido a cazar a ...,2025-03-29 03:21:31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3792,1jgdbz8,There's No Freaking Way I'll be Your Lover! Un...,2025-03-21 07:04:16,zenzen_0,https://www.watanare-anime.com/wordpress/wp-co...,207,,No,1519,1519,0,1jgdbz8,miy6ti3,Aerodynamic41,"Ah, Renako and her harem.\n\nI'm curious about...",2025-03-21 07:10:31
3793,1jgdbz8,There's No Freaking Way I'll be Your Lover! Un...,2025-03-21 07:04:16,zenzen_0,https://www.watanare-anime.com/wordpress/wp-co...,207,,No,1519,1519,0,1jgdbz8,miyguen,sidewinderaw11,"Ahh, the illustrator for the series also did S...",2025-03-21 08:36:10
3794,1jgdbz8,There's No Freaking Way I'll be Your Lover! Un...,2025-03-21 07:04:16,zenzen_0,https://www.watanare-anime.com/wordpress/wp-co...,207,,No,1519,1519,0,1jgdbz8,miy6ytv,schmurles,Ichikana in another yuri anime? Count me in. E...,2025-03-21 07:11:56
3795,1jgdbz8,There's No Freaking Way I'll be Your Lover! Un...,2025-03-21 07:04:16,zenzen_0,https://www.watanare-anime.com/wordpress/wp-co...,207,,No,1519,1519,0,1jgdbz8,miyb9aq,Kent93,"Mai VA is great, satsuki as well",2025-03-21 07:50:44


In [14]:
#dataframe solo de la publicacion
print(df_posts.head())

   id_post                                        titulo_post  \
0  1jtbb4w                        Andrei Sokolov en instagram   
1  1jmdog7  Cuando te digan que Chile es xenofobico muestr...   
2  1jp6c9x                         Cuando eres chileno y ....   
3  1jjjcak                   En qué quedó el tema del cabro ?   
4  1jg862n                                    Así con los TEA   

            fecha_post           autor_post  \
0  2025-04-06 22:51:52       extremoenpalta   
1  2025-03-29 00:40:52  Sensitive-Play-4770   
2  2025-04-01 17:44:21             ozarurai   
3  2025-03-25 10:27:54        Key_Self_5514   
4  2025-03-21 00:56:37     Ok_Recording8157   

                                    URL  numero_comentarios texto_post  \
0  https://i.redd.it/h9odhufhubte1.jpeg                 223              
1       https://v.redd.it/y9r623zsujre1                 153              
2   https://i.redd.it/vhey8xtccase1.png                 117              
3  https://i.redd.it/82z2v3x38

In [15]:
#dataframe solo de los comentarios
print(df_comentarios.head())

   id_post id_comentario      autor_comentario  \
0  1jtbb4w       mlt2gxd            edalgomezn   
1  1jtbb4w       mlt1eae  Own-Refrigerator7804   
2  1jtbb4w       mlszhso             Pajeringo   
3  1jtbb4w       mlt3y0q     rascame-los-cocos   
4  1jtbb4w       mlta37r          Morralronnic   

                                    texto_comentario     fecha_comentario  
0                   Más patriota que varios chilenos  2025-04-06 23:20:53  
1  Jajaja se detonó \nMe cae bien el Andrei, es m...  2025-04-06 23:13:09  
2                                     El ql los meó   2025-04-06 22:59:41  
3  >Comparando todo con Buenos Aires y búrlandose...  2025-04-06 23:31:43  
4  Era humillarlos no dejarlos en el piso, la uni...  2025-04-07 00:19:49  


### Exportar los tres dataframes creados 

In [16]:
#exportar dataset de publicaciones a un csv
df_posts.to_csv('posts.csv', index=False, encoding='utf-8')

In [17]:
##exportar dataset de comentarios a un csv
df_comentarios.to_csv('comentarios.csv', index=False, encoding='utf-8')

In [18]:
#exportar el dataset con los dataframe combinados
df_duckdb.to_csv('posts_comentarios_combinado_duckdb.csv', index=False, encoding='utf-8')

In [19]:
#saber el directorio actual para ver se guardaron los csv
import os
print(os.getcwd())

c:\Users\diego\Downloads


### Recomiendo no abrir directamente el csv , ya que al hacerlo asi en exel la informacion no se muestra ordenada, se debe abrir exel e ir a datos>obtener transformar datos>seleccionar el csv>cargar